In [1]:
import sys
!{sys.executable} -m pip install pandas
!{sys.executable} -m pip install pulp
!{sys.executable} -m pip install openpyxl
!{sys.executable} -m pip install cplex
!{sys.executable} -m pip install docplex

^C


In [1]:

import pandas as pd
import pulp
from datetime import timedelta

In [2]:
SAMPLE_DATA_FILE = "./data/output/sample_de/sample_dane_puste_przebiegi.xlsx"
DISTANCE_FILE = "./data/output/sample_de/sample_de_cities_distance_matrix.xlsx"
DISTANCE_FILE_SHEET = "Cities_Distances_Matrix"

TIME_FILE = "./data/output/sample_de/sample_de_cities_time_matrix.xlsx"
TIME_FILE_SHEET = "Cities_Time_Matrix"

## Load data

In [3]:
route_df = pd.read_excel(SAMPLE_DATA_FILE, index_col=0)

route_df = route_df.dropna()
route_df

,client,tractor,route,load_city,load_postal_code,load_country,unload_city,unload_postal_code,unload_country,pickup_planned,pickup_actual,delivery_planned,delivery_actual,empty_km_map,loaded_km_map,total_km_map
full_number,,,,,,,,,,,,,,,,
27070/2025,61,PO2PY63,(DE) Pritzwalk - (HU) Csomor,pritzwalk,16928,de,csömör,2141,hu,2025-09-08 07:00:00,2025-09-08 06:47:13,2025-09-09 07:30:00,2025-09-09 16:25:00,137.687,1180.231,1317.918
27356/2025,35,PO2PY63,(HU) Csömör - (DE) CRAILSHEIM,csömör,2141,hu,crailsheim,74564,de,2025-09-10 04:00:00,2025-09-10 14:14:18,2025-09-11 14:00:00,2025-09-11 15:25:00,3.267,898.686,901.953
27576/2025,61,PO2PY63,(DE) CRAILSHEIM - (DE) Landsberg,crailsheim,74564,de,landsberg,06188,de,2025-09-11 08:00:00,2025-09-11 15:25:14,2025-09-12 18:00:00,2025-09-15 08:00:00,0.000,383.061,383.061
27165/2025,35,PO2PY67,(FR) amiens - (DE) KIEL,amiens,80013,fr,kiel,24109,de,2025-09-08 17:00:00,2025-09-08 18:30:00,2025-09-10 06:00:00,2025-09-10 08:35:00,199.784,899.613,1099.397
27393/2025,47,PO2PY67,(DE) Hamburg - (PL) OSIECZNA,hamburg,21129,de,osieczna,64-113,pl,2025-09-10 09:00:00,2025-09-10 13:00:00,2025-09-11 18:00:00,2025-09-11 19:28:47,101.892,592.303,694.195
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
27283/2025,61,WPR1249U,(PL) Łódź - (DE) BERLIN,łódź,94-406,pl,berlin,12099,de,2025-09-09 17:00:00,2025-09-10 03:06:00,2025-09-10 06:00:00,2025-09-10 12:02:35,0.000,517.388,517.388
27479/2025,61,WPR1249U,(DE) BERLIN - (PL) Łódź,berlin,12099,de,łódź,94-406,pl,2025-09-10 10:00:00,2025-09-10 12:45:00,2025-09-10 19:00:00,2025-09-10 22:19:00,0.000,519.184,519.184
27485/2025,61,WPR1249U,(PL) Łódź - (DE) BERLIN,łódź,94-406,pl,berlin,12099,de,2025-09-10 21:00:00,2025-09-10 22:23:00,2025-09-11 10:00:00,2025-09-11 15:12:00,0.000,517.388,517.388


In [4]:
matrix_loaded_km_df = pd.read_excel(DISTANCE_FILE, index_col=0)

matrix_loaded_km_df = matrix_loaded_km_df.astype(float)

matrix_loaded_km_df

,agnadello,altfeld,alzenau,amiens,amt wachsenburg,andernach,andwil sg,ardon,aulnay sous bois,balve,...,worms,wörnitz,wörth am rhein,würzburg,xixona,zbaszynek,zgorzelec,zuzenhausen,zwijndrecht,łódź
city,,,,,,,,,,,,,,,,,,,,,
agnadello,0.00,689.70,740.73,1028.04,821.21,807.33,378.08,451.22,966.16,900.30,...,658.60,579.96,578.05,659.25,1506.68,1285.83,1089.07,628.25,983.05,1447.05
altfeld,689.70,0.00,66.21,667.41,233.55,206.16,403.82,624.66,638.72,278.81,...,144.04,160.91,217.04,42.11,1879.15,720.84,524.08,152.46,475.12,903.16
alzenau,740.73,66.21,0.00,632.98,261.09,171.73,466.34,599.54,604.29,221.53,...,109.61,223.43,182.61,104.63,1854.03,738.14,579.94,164.61,440.69,1004.03
amiens,1028.04,667.41,632.98,0.00,763.55,471.23,814.44,559.89,137.60,520.24,...,547.10,742.15,599.32,703.97,1658.59,1145.23,1082.40,603.24,257.55,1411.12
amt wachsenburg,821.21,233.55,261.09,763.55,0.00,309.84,540.47,798.59,803.84,333.12,...,308.66,292.82,381.66,181.76,2053.08,487.07,328.87,329.34,572.07,752.97
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
zbaszynek,1285.83,720.84,738.14,1145.23,487.07,788.50,971.12,1274.60,1211.10,674.51,...,785.68,688.04,858.81,669.58,2528.58,0.00,180.89,799.03,908.72,301.54
zgorzelec,1089.07,524.08,579.94,1082.40,328.87,630.30,774.36,1073.35,1137.23,616.82,...,627.48,491.28,672.40,472.82,2327.34,180.89,0.00,602.13,916.27,385.73
zuzenhausen,628.25,152.46,164.61,603.24,329.34,230.94,336.50,485.65,566.04,323.45,...,81.80,157.82,84.70,138.42,1739.63,799.03,602.13,0.00,479.61,981.94


In [5]:
matrix_loaded_time_df = pd.read_excel(TIME_FILE, index_col=0)

matrix_loaded_time_df = matrix_loaded_time_df.astype(float)

matrix_loaded_time_df

,agnadello,altfeld,alzenau,amiens,amt wachsenburg,andernach,andwil sg,ardon,aulnay sous bois,balve,...,worms,wörnitz,wörth am rhein,würzburg,xixona,zbaszynek,zgorzelec,zuzenhausen,zwijndrecht,łódź
city,,,,,,,,,,,,,,,,,,,,,
agnadello,0.00,7.26,7.41,10.48,8.58,8.01,4.79,5.77,9.57,9.07,...,6.80,6.62,6.24,7.10,14.64,11.88,10.24,6.56,10.20,13.71
altfeld,7.26,0.00,0.71,6.09,2.28,1.85,3.81,5.62,5.84,2.77,...,1.43,1.79,1.89,0.58,17.08,6.22,4.58,1.43,4.29,8.32
alzenau,7.41,0.71,0.00,5.85,2.34,1.61,4.32,5.49,5.60,2.35,...,1.19,2.30,1.65,1.09,16.95,6.40,4.97,1.60,4.05,8.69
amiens,10.48,6.09,5.85,0.00,7.12,4.60,7.78,5.43,1.54,5.29,...,5.09,6.97,5.44,6.41,15.89,10.20,9.75,5.58,2.80,12.48
amt wachsenburg,8.58,2.28,2.34,7.12,0.00,3.11,5.20,7.10,7.29,3.16,...,2.80,3.12,3.26,1.97,18.56,4.41,2.99,3.10,5.37,6.70
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
zbaszynek,11.88,6.22,6.40,10.20,4.41,7.18,8.50,11.10,10.64,6.06,...,6.84,6.13,7.31,5.92,22.52,0.00,2.65,7.06,7.91,3.06
zgorzelec,10.24,4.58,4.97,9.75,2.99,5.75,6.86,9.48,9.89,5.44,...,5.42,4.49,5.80,4.28,20.90,2.65,0.00,5.30,7.72,4.00
zuzenhausen,6.56,1.43,1.60,5.58,3.10,2.16,3.37,4.55,5.29,3.24,...,0.89,1.79,0.86,1.36,15.97,7.06,5.30,0.00,4.50,9.07


## Prepare solver model

In [6]:
### V2
# ------------------------
# 1. Dane
# ------------------------
# route_df, empty_runs_df, city_distance_df, city_time_df już wczytane

tractors = route_df['tractor'].unique()
delta_hours = 12        # dopuszczalne okno czasowe przy pustym przejeździe

max_idle_hours = 24 #60     # maksymalny dopuszczalny czas oczekiwania w godzinach (przejazdy oddalone od seibie o 2 dni nie sa uwzgledniane do listy kandydatów)
min_empty_km = 10       # minimalna odległość pustego przejazdu w km (poniżej tej wartości nie są uwzględniane do listy kandydatów)
max_empty_km = 500 # 300    # maksymalna odległość pustego przejazdu w km (powyżej tej wartości nie są uwzgledniane do listy kandydatów)

In [7]:

# ------------------------
# 2. Generowanie kandydatów pustych połączeń
# ------------------------

empty_candidates = []

route_df_sorted = route_df.sort_values('delivery_planned')

routes = route_df_sorted.reset_index()  # żeby mieć idx_i jako liczbę
n = len(routes)

for i in range(n):
    route_i = routes.loc[i]
    arrival_time = route_i['delivery_planned'] + pd.to_timedelta(matrix_loaded_time_df.loc[route_i['unload_city']], unit='h')
    
    # szukamy tylko tras, które mogą się zdarzyć w oknie czasowym
    for j in range(i+1, n):  # tylko późniejsze trasy w czasie
        route_j = routes.loc[j]
        travel_time_hours = matrix_loaded_time_df.loc[route_i['unload_city'], route_j['load_city']]
        travel_distance_km = matrix_loaded_km_df.loc[route_i['unload_city'], route_j['load_city']]

        if travel_distance_km < min_empty_km or travel_distance_km > max_empty_km:
            continue  # pomijamy zbyt krótkie puste przejazdy

        arrival_time = route_i['delivery_planned'] + pd.Timedelta(hours=travel_time_hours)
        idle_time = (route_j['pickup_planned'] - arrival_time).total_seconds() / 3600
        
        if idle_time < 0:
            continue    # rj zaczyna się przed przybyciem -> nie możliwe

        if idle_time > max_idle_hours:
            break  # dalsze trasy będą jeszcze później w czasie → nie pasują
        if 0 <= idle_time <= max_idle_hours and arrival_time <= route_j['pickup_planned'] + pd.Timedelta(hours=delta_hours):
            empty_candidates.append({
                'from_route': route_i['full_number'],
                'to_route': route_j['full_number'],
                'empty_km': travel_distance_km,
                'idle_hours': idle_time
            })

empty_candidates_df = pd.DataFrame(empty_candidates)


In [8]:
empty_candidates_df.columns

Index(['from_route', 'to_route', 'empty_km', 'idle_hours'], dtype='object')

In [8]:
# po dodaniu max_empty_km pozbyliśmy sie sytuacji rpzejazdów miedzy tymi samymi miastami - ten fragment nic nie wnosi juz
from_unload = route_df.loc[empty_candidates_df['from_route'], 'unload_city'].to_numpy()
to_load = route_df.loc[empty_candidates_df['to_route'], 'load_city'].to_numpy()

empty_candidates_df = empty_candidates_df[from_unload != to_load]

empty_candidates_df

,from_route,to_route,empty_km,idle_hours
0,27068/2025,27076/2025,75.67,6.94
1,27068/2025,26889/2025,75.67,6.94
2,27068/2025,26988/2025,75.67,1.94
3,27068/2025,26826/2025,75.67,0.94
4,27068/2025,27079/2025,339.38,1.55
...,...,...,...,...
5447,27801/2025,27710/2025,493.92,3.18
5448,27653/2025,27639/2025,140.60,1.48
5449,27653/2025,27710/2025,356.72,3.41
5450,27875/2025,27710/2025,275.13,3.95


In [9]:
# ------------------------
# 3. Tworzenie zmiennych decyzyjnych
# ------------------------
# x[(tractor, from_route, to_route)] = 1 jeśli traktor wykonuje pusty przejazd między trasami
index_list = [
    (t, row.from_route, row.to_route)
    for row in empty_candidates_df.itertuples(index=False)
    for t in tractors
]

x = pulp.LpVariable.dicts(
    "empty",
    index_list,
    cat='Binary'
)

In [10]:
# y[(tractor, route)] = 1 jeśli traktor wykonuje trasę pełną lub przejętą

index_list_y = [
    (t, r)
    for r in route_df.index
    for t in tractors
]

y = pulp.LpVariable.dicts(
    "full",
    index_list_y,
    cat='Binary'
)

In [ ]:
# # ------------------------
# # 4. Definicja problemu i funkcji celu
# # ------------------------
# prob = pulp.LpProblem("Minimize_Empty_Km", pulp.LpMinimize)

# # Pobieramy dane do macierzy numpy
# empty_km_array = empty_candidates_df[['from_route', 'to_route', 'empty_km']].to_numpy()

# # Funkcja celu: minimalizacja pustych km
# prob += pulp.lpSum(
#     empty_km * x[(t, fr, to_)]
#     for fr, to_, empty_km in empty_km_array
#     for t in tractors
# )



In [ ]:

# # ------------------------
# # 5. Ograniczenia
# # ------------------------

# # a) Każda pełna trasa musi być wykonana przez dokładnie jeden traktor
# for r in route_df.index:
#     prob += pulp.lpSum([y[(t, r)] for t in tractors]) == 1

# # b) Puste przejazdy mogą być wykonane tylko jeśli traktor wykonuje obie trasy
# # for _, row in empty_candidates_df.iterrows():
# #     for t in tractors:
# #         prob += x[(t, row['from_route'], row['to_route'])] <= y[(t, row['from_route'])]
# #         prob += x[(t, row['from_route'], row['to_route'])] <= y[(t, row['to_route'])]

# # for t in tractors:
# #     prob += pulp.lpSum(
# #         x[(t, row['from_route'], row['to_route'])] 
# #         for _, row in empty_candidates_df.iterrows()
# #     ) <= 1  # przykład sumaryczny, zależy od logiki

# # b) x możliwe tylko jeśli ten sam traktor robi obie trasy
# for _, row in empty_candidates_df.iterrows():
#     fr = row['from_route']
#     to = row['to_route']
#     for t in tractors:
#         prob += x[(t, fr, to)] <= y[(t, fr)]
#         prob += x[(t, fr, to)] <= y[(t, to)]


# # c) nie można użyć tego samego empty candidate więcej niż raz (przez różnych traktorów)
# for _, row in empty_candidates_df.iterrows():
#     fr = row['from_route']
#     to = row['to_route']
#     prob += pulp.lpSum(x[(t, fr, to)] for t in tractors) <= 1

In [ ]:
# # c) Sekwencja tras dla tractora: czasowe ograniczenia
# # jeśli pusty przejazd jest przypisany, musimy zachować czas
# M = (route_df['pickup_planned'].max() - route_df['delivery_planned'].min()).total_seconds() / 3600
# M


105.0

In [11]:
# s[(tractor, route)] = 1 jeśli traktor t ZACZYNA pracę od trasy r
index_list_s = [
    (t, r)
    for r in route_df.index
    for t in tractors
]

s = pulp.LpVariable.dicts(
    "start",
    index_list_s,
    cat='Binary'
)

In [ ]:
# ------------------------
# 4. Definicja problemu i funkcji celu (POPRAWIONA - CHAIN MAXIMALIZATION)
# ------------------------
prob = pulp.LpProblem("Minimize_Empty_Km_With_Chain_Reward", pulp.LpMinimize)

# Definiujemy stałą nagrodę za połączenie dwóch tras. 
# Musi być większa niż najdłuższy możliwy pusty przejazd (np. 1000 lub 10000).
# CHAIN_REWARD = 1000

# Pobieramy dane do macierzy numpy dla szybszego dostępu
empty_km_array = empty_candidates_df[['from_route', 'to_route', 'empty_km']].to_numpy()

# # Funkcja celu: 
# # Sumujemy (Dystans * x) - (Nagroda * x).
# # Ponieważ minimalizujemy, solver będzie dążył do ustawienia x=1 (zyskując -10000 do wyniku),
# # wybierając przy tym x o najmniejszym 'empty_km'.
# prob += pulp.lpSum(
#     (empty_km - CHAIN_REWARD) * x[(t, fr, to_)]
#     for fr, to_, empty_km in empty_km_array
#     for t in tractors
# )

## II -> Zamiast rewarda → mały epsilon (jednak bez epsilonu)

prob += pulp.lpSum(
    empty_km * x[(t, fr, to_)]
    for fr, to_, empty_km in empty_km_array
    for t in tractors
)

🔍 Co ten model gwarantuje (ważne)

✔️ każda trasa wykonana dokładnie raz

✔️ żaden traktor nie jest klonowany

✔️ każda trasa:

albo jest startem

albo ma dokładnie jeden pusty dojazd

✔️ puste km są liczone i karane

✔️ nie da się uzyskać „0 km pustych” sztucznie

✔️ solver musi szukać realnych chainów

In [13]:
# ------------------------
# 5. Ograniczenia
# ------------------------

# Budowa list łuków
outgoing_arcs = {r: [] for r in route_df.index}
incoming_arcs = {r: [] for r in route_df.index}

for row in empty_candidates_df.itertuples(index=False):
    outgoing_arcs[row.from_route].append(row.to_route)
    incoming_arcs[row.to_route].append(row.from_route)

# (a) Każda trasa musi być wykonana dokładnie raz
for r in route_df.index:
    prob += pulp.lpSum(y[(t, r)] for t in tractors) == 1

# (b) Każdy traktor może zacząć tylko raz
for t in tractors:
    prob += pulp.lpSum(s[(t, r)] for r in route_df.index) <= 1

# (c) Flow conservation + brak teleportacji
for t in tractors:
    for r in route_df.index:

        # maksymalnie jeden wyjazd pusty z trasy
        if outgoing_arcs[r]:
            prob += (
                pulp.lpSum(x[(t, r, r2)] for r2 in outgoing_arcs[r])
                <= y[(t, r)]
            )

        # dokładnie: start ALBO jeden dojazd pusty
        prob += (
            s[(t, r)] +
            pulp.lpSum(x[(t, r1, r)] for r1 in incoming_arcs[r])
            ==
            y[(t, r)]
        )

# (d) Ograniczenia czasowe (Big-M)
M = (route_df['pickup_planned'].max() - route_df['delivery_planned'].min()).total_seconds() / 3600

from_routes = route_df.loc[empty_candidates_df['from_route']]
to_routes   = route_df.loc[empty_candidates_df['to_route']]

travel_times = [
    matrix_loaded_time_df.loc[uf, tl]
    for uf, tl in zip(from_routes['unload_city'], to_routes['load_city'])
]

A_times = from_routes['delivery_planned'].to_numpy() + pd.to_timedelta(travel_times, unit='h')
B_times = to_routes['pickup_planned'].to_numpy() + pd.to_timedelta(delta_hours, unit='h')

A_num = (A_times.astype('int64') / 3.6e12).tolist()
B_num = (B_times.astype('int64') / 3.6e12).tolist()

for i, (fr, tr) in enumerate(zip(
    empty_candidates_df['from_route'],
    empty_candidates_df['to_route']
)):
    for t in tractors:
        prob += A_num[i] <= B_num[i] + M * (1 - x[(t, fr, tr)])


In [14]:
# Sprawdzenie wielkości modelu
n_vars = prob.numVariables()
n_constrs = prob.numConstraints()

print(f"Liczba zmiennych decyzyjnych: {n_vars}")
print(f"Liczba ograniczeń: {n_constrs}")

Liczba zmiennych decyzyjnych: 787968
Liczba ograniczeń: 784736


In [ ]:
# from_routes = route_df.loc[empty_candidates_df['from_route']]
# to_routes = route_df.loc[empty_candidates_df['to_route']]

# travel_times = [
#     matrix_loaded_time_df.loc[uf, tl]
#     for uf, tl in zip(from_routes['unload_city'], to_routes['load_city'])
# ]

# A_times = from_routes['delivery_planned'].to_numpy() + pd.to_timedelta(travel_times, unit='h')
# B_times = to_routes['pickup_planned'].to_numpy() + pd.to_timedelta(delta_hours, unit='h')

# A_num_list = (A_times.astype('int64') / 3.6e12).tolist()
# B_num_list = (B_times.astype('int64') / 3.6e12).tolist()

# # ^^^ wektoryzowanie dla szybszego działania ^^^
# # -------------------------------

# from_routes_list = empty_candidates_df['from_route'].to_list()
# to_routes_list = empty_candidates_df['to_route'].to_list()

# for i in range(len(empty_candidates_df)):
#     fr = from_routes_list[i]
#     tr = to_routes_list[i]
#     A_num = A_num_list[i]
#     B_num = B_num_list[i]

#     for t in tractors:
#         prob += A_num <= B_num + M * (1 - x[(t, fr, tr)])

✔️ CO MÓWI MODEL

Każda pełna trasa ma być wykonana przez dokładnie 1 traktor
→ y[t, r] = 1 dla dokładnie jednego traktora

Pusty przejazd x[t, A, B] może być tylko wtedy, gdy
traktor wykonuje obie trasy: A i B

Big-M wymusza tylko to:
jeśli x[t,A,B] = 1 → dostawa(A) + przejazd ≤ załadunek(B)

In [15]:
# Support function to fill empty_km_map in new df
def fill_empty_km(df, distance_matrix):
    for tractor, group in df.groupby('tractor'):
        idxs = group.index.tolist()

        for i in range(1, len(idxs)):
            prev_idx = idxs[i - 1]
            curr_idx = idxs[i]
            prev_unload = df.loc[prev_idx, 'unload_city']
            curr_load = df.loc[curr_idx, 'load_city']
            # pobieramy dystans z matrix_loaded_km_df
            empty_km = distance_matrix.loc[prev_unload, curr_load]
            df.loc[curr_idx, 'empty_km_map'] = empty_km
    return df

### b) GLPK

In [28]:
# ------------------------
# 6. Rozwiązanie problemu
# ------------------------
prob.solve(pulp.GLPK_CMD(
    path="C:\\Program Files\\glpk-4.65\\w64\\glpsol.exe",
    msg=True,
    options=["--tmlim", "3600"]
))

print(f"Status rozwiązania: {pulp.LpStatus[prob.status]}")
print(f"Minimalna suma pustych km: {pulp.value(prob.objective):.2f} km\n")

Status rozwiązania: Undefined
Minimalna suma pustych km: 0.00 km



In [29]:
# ------------------------
# 7. Wyświetlenie wyników
# ------------------------
for _, row in empty_candidates_df.iterrows():
    for t in tractors:
        var = x[(t, row['from_route'], row['to_route'])]
        if var.varValue == 1:
            from_route = route_df.loc[row['from_route']]
            to_route = route_df.loc[row['to_route']]
            print(f"Tractor {t}: pusty z {from_route['unload_city']} -> {to_route['load_city']} ({row['empty_km']} km)")


In [34]:

new_routes_glpk = []

for t in tractors:
    for r in route_df.sort_values('delivery_planned').index:
        if y[(t, r)].varValue == 1:
            route = route_df.loc[r]
            new_routes_glpk.append({
                'full_number': r,
                'tractor': t,
                'load_city': route['load_city'],
                'unload_city': route['unload_city'],
                'pickup_planned': route['pickup_planned'],
                'pickup_actual': route['pickup_actual'],
                'delivery_planned': route['delivery_planned'],
                'delivery_actual': route['delivery_actual'],
                'loaded_km_map': route['loaded_km_map'],
                'empty_km_map': 0,
            })
            print(f"Tractor {t}: pełna trasa {route['load_city']} -> {route['unload_city']} od {route['pickup_planned']} do {route['delivery_planned']}")

new_routes_glpk_df = pd.DataFrame(new_routes_glpk)

In [35]:
new_routes_glpk_df.head()

""


In [32]:
new_routes_glpk_df['empty_km_map'] = new_routes_glpk_df['empty_km_map'].astype(float)
new_routes_glpk_df = fill_empty_km(new_routes_glpk_df, matrix_loaded_km_df)

# # zapis do Excela
new_routes_glpk_df.to_excel("./data/output/sample_de/sample_dane_puste_przebiegi_optimalized_glpk_13_12.xlsx", index=False)
new_routes_glpk_df

KeyError: 'empty_km_map'

In [36]:
# NOWE OBLCZENIE  - Suma empty_km_map w new_routes_df
total_empty_new_routes = new_routes_glpk_df['empty_km_map'].sum()
print(f"Suma empty_km_map w new_route_df: {total_empty_new_routes} km")

KeyError: 'empty_km_map'

In [26]:
# Suma empty_km_map w new_routes_df
total_empty_new_routes = new_routes_glpk_df['empty_km_map'].sum()
print(f"Suma empty_km_map w new_route_df: {total_empty_new_routes} km")

Suma empty_km_map w new_route_df: 189585.82 km


### c) CPLEX (IBM)

In [16]:
# Ustawiamy limit czasu na 600 sekund (10 min) i akceptowalny błąd 5% (0.05)
# To pozwoli solverowi zwrócić wynik, gdy tylko znajdzie coś sensownego, zamiast liczyć w nieskończoność.
prob.solve(pulp.CPLEX_PY(msg=True, timeLimit=3600, gapRel=0.05))

print(f"Status rozwiązania: {pulp.LpStatus[prob.status]}")
print(f"Minimalna suma pustych km: {pulp.value(prob.objective):.2f} km\n")

Version identifier: 22.1.1.0 | 2022-11-27 | 9160aff4d
CPXPARAM_Read_DataCheck                          1
CPXPARAM_TimeLimit                               3600
CPXPARAM_MIP_Tolerances_MIPGap                   0.050000000000000003
Tried aggregator 2 times.
MIP Presolve eliminated 697856 rows and 0 columns.
Aggregator did 25472 substitutions.
Reduced MIP has 61408 rows, 762496 columns, and 1614080 nonzeros.
Reduced MIP has 762496 binaries, 0 generals, 0 SOSs, and 0 indicators.
Presolve time = 3,28 sec. (2084,71 ticks)
Tried aggregator 1 time.
Detecting symmetries...
Reduced MIP has 61408 rows, 762496 columns, and 1614080 nonzeros.
Reduced MIP has 762496 binaries, 0 generals, 0 SOSs, and 0 indicators.
Presolve time = 4,27 sec. (2276,81 ticks)
Probing time = 5,36 sec. (262,22 ticks)
Clique table members: 917586.
MIP emphasis: balance optimality and feasibility.
MIP search method: dynamic search.
Parallel mode: deterministic, using up to 20 threads.
Root relaxation solution time = 19,88 sec.

In [17]:
# ------------------------
# 7. Wyświetlenie wyników
# ------------------------
for _, row in empty_candidates_df.iterrows():
    for t in tractors:
        var = x[(t, row['from_route'], row['to_route'])]
        if var.varValue == 1:
            from_route = route_df.loc[row['from_route']]
            to_route = route_df.loc[row['to_route']]
            print(f"Tractor {t}: pusty z {from_route['unload_city']} -> {to_route['load_city']} ({row['empty_km']} km)")


Tractor WGM8278K: pusty z andernach -> euskirchen (75.67 km)
Tractor PZ4R705: pusty z frankenthal -> strasbourg (134.79 km)
Tractor PO8PX23: pusty z amiens -> la sentinelle (122.95 km)
Tractor PO2PY67: pusty z euskirchen -> dortmund (134.29 km)
Tractor WGM5124H: pusty z euskirchen -> dortmund (134.29 km)
Tractor WGM88452: pusty z łódź -> beeskow (408.16 km)
Tractor WGM1643M: pusty z löbichau -> großsteinberg (118.24 km)
Tractor PO2PY72: pusty z euskirchen -> crailsheim (369.52 km)
Tractor WGM8368L: pusty z crailsheim -> nürnberg (106.62 km)
Tractor WGM8717L: pusty z nürnberg -> crailsheim (106.62 km)
Tractor PO8PX24: pusty z crailsheim -> altfeld (140.6 km)
Tractor PZ4R802: pusty z crailsheim -> andwil sg (285.47 km)
Tractor PY85916: pusty z lomme -> trith-saint-léger (62.07 km)
Tractor PY13051: pusty z hénin-beaumont -> amiens (117.53 km)
Tractor WGM1647M: pusty z euskirchen -> herne (124.7 km)
Tractor PY15618: pusty z berlin -> pritzwalk (141.88 km)
Tractor WGM1648M: pusty z berlin -

In [18]:

new_routes_cplex = []

for t in tractors:
    for r in route_df.sort_values('delivery_planned').index:
        if y[(t, r)].varValue == 1:
            route = route_df.loc[r]
            new_routes_cplex.append({
                'full_number': r,
                'tractor': t,
                'load_city': route['load_city'],
                'unload_city': route['unload_city'],
                'pickup_planned': route['pickup_planned'],
                'pickup_actual': route['pickup_actual'],
                'delivery_planned': route['delivery_planned'],
                'delivery_actual': route['delivery_actual'],
                'loaded_km_map': route['loaded_km_map'],
                'empty_km_map': 0,
            })
            print(f"Tractor {t}: pełna trasa {route['load_city']} -> {route['unload_city']} od {route['pickup_planned']} do {route['delivery_planned']}")

new_routes_cplex_df = pd.DataFrame(new_routes_cplex)

Tractor PO2PY63: pełna trasa euskirchen -> amiens od 2025-09-08 16:00:00 do 2025-09-09 14:00:00
Tractor PO2PY63: pełna trasa trith-saint-léger -> frechen od 2025-09-09 16:00:00 do 2025-09-10 08:00:00
Tractor PO2PY63: pełna trasa euskirchen -> dintikon od 2025-09-10 10:00:00 do 2025-09-11 11:00:00
Tractor PO2PY67: pełna trasa crailsheim -> euskirchen od 2025-09-08 13:00:00 do 2025-09-08 15:45:00
Tractor PO2PY67: pełna trasa dortmund -> tastrup od 2025-09-08 22:00:00 do 2025-09-10 02:00:00
Tractor PO2PY67: pełna trasa lübeck -> dortmund od 2025-09-10 10:30:00 do 2025-09-11 06:00:00
Tractor PO2PY67: pełna trasa euskirchen -> amiens od 2025-09-12 04:00:00 do 2025-09-12 14:00:00
Tractor PO2PY71: pełna trasa csömör -> crailsheim od 2025-09-08 09:00:00 do 2025-09-09 16:00:00
Tractor PO2PY71: pełna trasa wörnitz -> berlin od 2025-09-09 18:45:00 do 2025-09-10 14:30:00
Tractor PO2PY71: pełna trasa pritzwalk -> crailsheim od 2025-09-11 09:00:00 do 2025-09-12 05:45:00
Tractor PO2PY72: pełna trasa 

In [19]:
new_routes_cplex_df['empty_km_map'] = new_routes_cplex_df['empty_km_map'].astype(float)
new_routes_cplex_df = fill_empty_km(new_routes_cplex_df, matrix_loaded_km_df)

# # zapis do Excela
new_routes_cplex_df.to_excel("./data/output/sample_de/sample_dane_puste_przebiegi_optimalized_cplex-single_stage.xlsx", index=False)
new_routes_cplex_df

,full_number,tractor,load_city,unload_city,pickup_planned,pickup_actual,delivery_planned,delivery_actual,loaded_km_map,empty_km_map
0,27008/2025,PO2PY63,euskirchen,amiens,2025-09-08 16:00:00,2025-09-08 20:19:00,2025-09-09 14:00:00,2025-09-09 14:05:00,406.308,0.00
1,27319/2025,PO2PY63,trith-saint-léger,frechen,2025-09-09 16:00:00,2025-09-09 16:10:00,2025-09-10 08:00:00,2025-09-10 10:55:00,275.989,123.09
2,27427/2025,PO2PY63,euskirchen,dintikon,2025-09-10 10:00:00,2025-09-10 12:54:00,2025-09-11 11:00:00,2025-09-11 14:10:00,569.677,34.33
3,27108/2025,PO2PY67,crailsheim,euskirchen,2025-09-08 13:00:00,2025-09-08 13:00:00,2025-09-08 15:45:00,2025-09-08 20:03:00,360.361,0.00
4,26971/2025,PO2PY67,dortmund,tastrup,2025-09-08 22:00:00,2025-09-08 13:57:58,2025-09-10 02:00:00,2025-09-10 04:00:00,647.606,134.29
...,...,...,...,...,...,...,...,...,...,...
347,27071/2025,WGM9818L,gattatico,crailsheim,2025-09-08 11:00:00,2025-09-08 11:00:00,2025-09-09 13:00:00,2025-09-09 12:00:00,795.672,0.00
348,27177/2025,WGM9818L,altfeld,crailsheim,2025-09-09 19:00:00,2025-09-09 21:09:00,2025-09-10 11:00:00,2025-09-10 09:19:00,143.640,140.60
349,27329/2025,WGM9818L,altfeld,crailsheim,2025-09-10 19:00:00,2025-09-10 15:52:22,2025-09-11 10:00:00,2025-09-11 10:09:00,143.640,140.60
350,27679/2025,WGM9818L,uffenheim,pampow,2025-09-11 12:00:00,2025-09-11 16:55:24,2025-09-12 08:00:00,2025-09-12 18:28:05,641.453,62.49


In [20]:
# Suma empty_km_map w new_routes_df
total_empty_new_routes = new_routes_cplex_df['empty_km_map'].sum()
print(f"Suma empty_km_map w new_route_df: {total_empty_new_routes} km")

Suma empty_km_map w new_route_df: 29247.850000000002 km


### Init data - empty passages sum (in km)

In [21]:
# użycie funkcji
route_df['empty_km_map'] = route_df['empty_km_map'].astype(float)
route_df = fill_empty_km(route_df, matrix_loaded_km_df)

# Suma empty_km_map w routes_df
# Jeśli routes_df nie ma kolumny 'empty_km_map', tworzymy ją z zerami
if 'empty_km_map' not in route_df.columns:
    route_df['empty_km_map'] = 0

total_empty_routes = route_df['empty_km_map'].sum()
print(f"Suma empty_km_map w route_df: {total_empty_routes} km")

Suma empty_km_map w route_df: 31293.294 km


## Tutorials & Sources

0. Install GLPK solver engine - https://sourceforge.net/projects/winglpk/

1. cplex (ibm) dpownlaod / instalation and python connection (requires python 10 :< ) https://www.youtube.com/watch?v=q7iVDGartf4&list=PLaxOs-8sLebuytu-pPSM4mtsR5VVlFtyW

2. Assecc to IBM for students - https://www.youtube.com/watch?v=HQcrzYqV9qo
3. links https://community.ibm.com/community/user/blogs/xavier-nodet1/2020/07/09/cplex-free-for-students?CommunityKey=ab7de0fd-6f43-47a9-8261-33578a231bb7&tab=
4. access to IBM applications https://academic.ibm.com/a2mt/downloads#/ via user email
4.2 IBM Porfile - https://www.ibm.com/ibmweb/myibm/us-en/profile


5. download pip for python 10 https://stackoverflow.com/questions/73029329/install-pip-on-window-10-pc


6. OInstall dependecies - conenct python 10 with CPLEX - "python D:\Program Files\IBM\ILOG\CPLEX_Studio2211\python\setup.py install"
